In [7]:
from rplidar import RPLidar
import numpy as np
import pyautogui
import time
from playsound import playsound
import subprocess
import tkinter as tk
from tkinter import *
from PIL import Image
from PIL import ImageTk,Image
from threading import Thread
import math

def new_window():
    print("New window")
    child = tk.Toplevel(win)
    child.title('New window')
    child.geometry("1000x800")
    child.configure(bg='White')
    child.state('zoomed')
    
    y_height = 9
    x_width = 22
    
    button1 = tk.Button(child, text='Apple', font=('Times New Roman', 20, 'bold'), height=y_height, width=x_width, command=action1, bg='Red', fg='White')
    button1.place(relx=0.2,rely=0.49,anchor = S)
    
    button2 = tk.Button(child, text='Balloons', font=('Times New Roman', 20, 'bold'), height=y_height, width=x_width, command=action2, bg='Blue', fg='White')
    button2.place(relx=0.5,rely=0.49,anchor = S)
    
    button3 = tk.Button(child, text='Birds', font=('Times New Roman', 20, 'bold'), height=y_height, width=x_width, command=action3, bg='Yellow', fg='White')
    button3.place(relx=0.8,rely=0.49,anchor = S)
    
    button4 = tk.Button(child, text='Building', font=('Times New Roman', 20, 'bold'), height=y_height, width=x_width, command=action4, bg='Gray', fg='White')
    button4.place(relx=0.2,rely=0.51,anchor = N)
    
    button5 = tk.Button(child, text='Flamingo', font=('Times New Roman', 20, 'bold'), height=y_height, width=x_width, command=action5, bg='Pink', fg='White')
    button5.place(relx=0.5,rely=0.51,anchor = N)
    
    button6 = tk.Button(child, text='Road', font=('Times New Roman', 20, 'bold'), height=y_height, width=x_width, command=action6, bg='Green', fg='White')
    button6.place(relx=0.8,rely=0.51,anchor = N)
    
    child.after(1000, lambda: calibrate())
    
    
    
def calibrate():
    print("Calibrate")
    i = 4
    while(i!=0):
        
        flag = 0
        q = 0
        
        if (i==4):
            print('Place pointer at top left corner')
        elif (i==3):
            print('Place pointer at top right corner')
        elif (i==2):
            print('Place pointer at bottom right corner')
        elif (i==1):
            print('Place pointer at bottom left corner')
            
        print('Keep the pointer in position for 5 secs')
        time.sleep(3)

        for q, scan in enumerate(lidar.iter_scans()):
            for j in range(len(scan)):
                if (scan[j-1][2]-scan[j][2] > 800) and (scan[j][1] > 160 and scan[j][1] < 210):
                    flag = 1
                    r = scan[j][2]
                    if scan[j][1] < 180:
                        angle = np.radians(scan[j][1]-90)
                        x = r * np.cos(angle)
                        y = r * np.sin(angle)
                        break;
                    elif scan[j][1] > 180:
                        angle = np.radians(270 - scan[j][1])
                        x = r * np.cos(angle)
                        y = r * np.sin(angle)
                        break;
                else:
                    flag = 0

            if q == 1:
                break;
                
        lidar.stop();
        
        if (i==4) and (flag==1):
            corners[0][0] = x
            corners[0][1] = y
            i = i- 1
            playsound('ping.wav')
            pyautogui.moveTo(960, 540)
        elif (i==3) and (flag==1):
            corners[1][0] = x
            corners[1][1] = y
            i = i- 1
            playsound('ping.wav')
        elif (i==2) and (flag==1):
            corners[2][0] = x
            corners[2][1] = y
            i = i- 1
            playsound('ping.wav')
        elif (i==1) and (flag==1):
            corners[3][0] = x
            corners[3][1] = y
            i = i- 1
            playsound('ping.wav')
        else:
            print('Calibration error')
        
        

    print(corners)
    lidar.stop()
    
    Thread(target = running).start()

def running():
    print("Running")
    A = np.mean([corners[1][0]+corners[0][0],corners[2][0]+corners[3][0]])
    B = np.mean([corners[0][1]-corners[3][1],corners[1][1]-corners[2][1]])
    
    
    data = np.zeros((5,2))
    flag = 0
    k = 0
    
    for i, scan in enumerate(lidar.iter_scans()):
        print('%d: Got %d measurments' % (i, len(scan)))
        distance = []
        for j in range(len(scan)):
            if (scan[j-1][2]-scan[j][2] > 800) and (scan[j][1] > 160 and scan[j][1] < 210):
                
                r = scan[j][2]
                
                if scan[j][1] < 180:
                    angle = np.radians(scan[j][1]-90)
                    x = r * np.cos(angle)
                    y = r * np.sin(angle)
                    a = corners[0][0] - x
                    b = corners[0][1] - y
                    per_x = (a/A)
                    per_y = (b/B)
                    screen_x = per_x * 1920
                    screen_y = per_y * 1080

                elif scan[j][1] > 180:
                    angle = np.radians(270 - scan[j][1])
                    x = r * np.cos(angle)
                    y = r * np.sin(angle)
                    a = corners[0][0] + x
                    b = corners[0][1] - y
                    per_x = (a/A)
                    per_y = (b/B)
                    screen_x = per_x * 1920
                    screen_y = per_y * 1080

                if (screen_x>1910):
                    screen_x = 1910
                if (screen_y>1070):
                    screen_y = 1070
                
                if (k == 5):
                    k = 0
                    avg_x = np.mean([data[0][0],data[1][0],data[2][0],data[3][0],data[4][0]])
                    avg_y = np.mean([data[0][1],data[1][1],data[2][1],data[3][1],data[4][1]])
                    avg_x = math.trunc(avg_x)
                    avg_y = math.trunc(avg_y)
                    for m in range(4):
                        if (abs(abs(data[m][0])-abs(data[m+1][0]))>100 or abs(abs(data[m][1])-abs(data[m+1][1]))>100):
                            flag = 1
                    if (flag != 1):
                        pyautogui.moveTo(avg_x, avg_y)
                        print("X: "+str(avg_x)+", Y: "+str(avg_y))
                        print(data)
                        pyautogui.click()
                        flag = 0
                        
                data[k][0] = screen_x;
                data[k][1] = screen_y;
                flag = 0
                k = k + 1
                break;

        if i > 1000:
            break 

    lidar.stop()

def action1():
    img_win = tk.Toplevel(win)
    img_win.title('Apple')
    img_win.geometry("300x300")
    img_win.configure(bg='White')
    img_win.state('zoomed')
    
    image1 = Image.open(r"C:\Users\shrik\Udacity_CVND\images\apple.jpg")
    test = ImageTk.PhotoImage(image1)
    label1 =Label(img_win, image=test)
    label1.image = test
    label1.place(relx=0.5, rely=0.5, anchor=CENTER)
    img_win.after(5000, lambda: img_win.destroy())
    
def action2():
    img_win = tk.Toplevel(win)
    img_win.title('Balloons')
    img_win.geometry("500x500")
    img_win.configure(bg='White')
    img_win.state('zoomed')
    
    image1 = Image.open(r"C:\Users\shrik\Udacity_CVND\images\ballons.jpg")
    test = ImageTk.PhotoImage(image1)
    label1 =Label(img_win, image=test)
    label1.image = test
    label1.place(relx=0.5, rely=0.5, anchor=CENTER)
    img_win.after(5000, lambda: img_win.destroy())
    
def action3():
    img_win = tk.Toplevel(win)
    img_win.title('Birds')
    img_win.geometry("500x500")
    img_win.configure(bg='White')
    img_win.state('zoomed')
    
    image1 = Image.open(r"C:\Users\shrik\Udacity_CVND\images\birds.jpg")
    test = ImageTk.PhotoImage(image1)
    label1 =Label(img_win, image=test)
    label1.image = test
    label1.place(relx=0.5, rely=0.5, anchor=CENTER)
    img_win.after(5000, lambda: img_win.destroy())
    
def action4():
    img_win = tk.Toplevel(win)
    img_win.title('Building')
    img_win.geometry("500x500")
    img_win.configure(bg='White')
    img_win.state('zoomed')
    
    image1 = Image.open(r"C:\Users\shrik\Udacity_CVND\images\city_hall.jpg")
    test = ImageTk.PhotoImage(image1)
    label1 =Label(img_win, image=test)
    label1.image = test
    label1.place(relx=0.5, rely=0.5, anchor=CENTER)
    img_win.after(5000, lambda: img_win.destroy())
    
def action5():
    img_win = tk.Toplevel(win)
    img_win.title('Flamingos')
    img_win.geometry("500x500")
    img_win.configure(bg='White')
    img_win.state('zoomed')
    
    image1 = Image.open(r"C:\Users\shrik\Udacity_CVND\images\flamingos.jpg")
    test = ImageTk.PhotoImage(image1)
    label1 =Label(img_win, image=test)
    label1.image = test
    label1.place(relx=0.5, rely=0.5, anchor=CENTER)
    img_win.after(5000, lambda: img_win.destroy())
    
def action6():
    img_win = tk.Toplevel(win)
    img_win.title('Road')
    img_win.geometry("500x500")
    img_win.configure(bg='White')
    img_win.state('zoomed')
    
    image1 = Image.open(r"C:\Users\shrik\Udacity_CVND\images\curved_lane.jpg")
    test = ImageTk.PhotoImage(image1)
    label1 =Label(img_win, image=test)
    label1.image = test
    label1.place(relx=0.5, rely=0.5, anchor=CENTER)
    img_win.after(5000, lambda: img_win.destroy())

if __name__ == '__main__':
    
    lidar = RPLidar('COM8',115200)
    corners = np.zeros((4,2))
    win = tk.Tk()
    win.title('Lidar based smart wall')
    win.geometry("1000x800")
    win.configure(bg='White')
    win.state('zoomed')

    #start label
    start_label = Label(win, text='Smart Wall',font=('Times New Roman', 18, 'bold'), bg='White')
    start_label.place(relx=0.5, rely=0.4, anchor=CENTER)

    #start button
    start_button = tk.Button(win, text='Start', font=('Times New Roman', 16, 'bold'), width=15, command=Thread(target = new_window).start, bg='Green', fg='White')
    start_button.place(relx=0.5, rely=0.5, anchor=CENTER)

    win.mainloop()

New window
Calibrate
Place pointer at top left corner
Keep the pointer in position for 5 secs
Place pointer at top right corner
Keep the pointer in position for 5 secs
Place pointer at bottom right corner
Keep the pointer in position for 5 secs
Place pointer at bottom left corner
Keep the pointer in position for 5 secs
[[ 263.03139728 1579.75211158]
 [ 409.36882263 1563.54764224]
 [ 369.85094455 1207.37228779]
 [ 330.09548548 1249.63723555]]
Running
0: Got 77 measurments
1: Got 114 measurments
2: Got 114 measurments
3: Got 112 measurments
4: Got 115 measurments
5: Got 115 measurments
6: Got 114 measurments
7: Got 114 measurments
8: Got 116 measurments
9: Got 116 measurments
10: Got 117 measurments
11: Got 116 measurments
12: Got 120 measurments
13: Got 116 measurments
14: Got 116 measurments
15: Got 116 measurments
16: Got 116 measurments
17: Got 116 measurments
18: Got 116 measurments
19: Got 115 measurments
20: Got 114 measurments
21: Got 116 measurments
22: Got 115 measurments
23: G

Too many measurments in the input buffer: 536/500. Clearing buffer...


X: 1183, Y: 724
[[1201.48006695  736.466242  ]
 [1183.23742594  731.60307878]
 [1179.64438069  716.07285186]
 [1178.66682101  725.4506782 ]
 [1172.50074921  714.3463413 ]]
196: Got 112 measurments
197: Got 113 measurments
198: Got 115 measurments
199: Got 110 measurments
200: Got 111 measurments
201: Got 113 measurments
202: Got 114 measurments
203: Got 115 measurments
204: Got 116 measurments
205: Got 114 measurments
206: Got 115 measurments
207: Got 117 measurments
208: Got 116 measurments
209: Got 115 measurments
210: Got 114 measurments
211: Got 114 measurments
212: Got 115 measurments
213: Got 117 measurments
214: Got 115 measurments
215: Got 115 measurments
216: Got 112 measurments
217: Got 118 measurments
218: Got 116 measurments
219: Got 115 measurments
220: Got 116 measurments
221: Got 118 measurments
222: Got 115 measurments
223: Got 115 measurments
224: Got 115 measurments
225: Got 115 measurments
226: Got 115 measurments
227: Got 114 measurments
228: Got 115 measurments
229

Too many measurments in the input buffer: 536/500. Clearing buffer...


X: 249, Y: 676
[[251.91034616 663.75005515]
 [249.46897928 660.9253747 ]
 [263.97431166 696.14862287]
 [245.52074901 644.8335488 ]
 [238.85501518 715.62331294]]
306: Got 115 measurments
307: Got 116 measurments
308: Got 117 measurments
309: Got 117 measurments
310: Got 116 measurments
311: Got 118 measurments
312: Got 114 measurments
313: Got 117 measurments
314: Got 117 measurments
315: Got 116 measurments
316: Got 118 measurments
317: Got 116 measurments
318: Got 115 measurments
319: Got 115 measurments
320: Got 119 measurments
321: Got 120 measurments
322: Got 116 measurments
323: Got 116 measurments
324: Got 115 measurments
325: Got 117 measurments
326: Got 116 measurments
327: Got 116 measurments
328: Got 113 measurments
329: Got 114 measurments
330: Got 114 measurments
331: Got 116 measurments
332: Got 115 measurments
333: Got 115 measurments
334: Got 115 measurments
335: Got 115 measurments
336: Got 116 measurments
337: Got 116 measurments
338: Got 115 measurments
339: Got 114 m

...

619: Got 113 measurments
620: Got 112 measurments
621: Got 113 measurments
622: Got 112 measurments
623: Got 110 measurments
624: Got 115 measurments
625: Got 116 measurments
626: Got 114 measurments
627: Got 116 measurments
628: Got 114 measurments
629: Got 115 measurments
630: Got 113 measurments
631: Got 116 measurments
632: Got 117 measurments
633: Got 115 measurments
634: Got 119 measurments
635: Got 115 measurments
636: Got 116 measurments
637: Got 115 measurments
638: Got 114 measurments
639: Got 113 measurments
640: Got 117 measurments
641: Got 114 measurments
642: Got 113 measurments
643: Got 117 measurments
644: Got 115 measurments
645: Got 116 measurments
646: Got 117 measurments
647: Got 115 measurments
648: Got 116 measurments
649: Got 116 measurments
650: Got 119 measurments
651: Got 118 measurments
652: Got 114 measurments
653: Got 117 measurments
654: Got 117 measurments
655: Got 117 measurments
656: Got 117 measurments
657: Got 113 measurments
658: Got 114 measurments


Too many measurments in the input buffer: 534/500. Clearing buffer...


744: Got 113 measurments
745: Got 113 measurments
746: Got 113 measurments
747: Got 113 measurments
748: Got 115 measurments
749: Got 114 measurments
750: Got 112 measurments
751: Got 116 measurments
752: Got 116 measurments
753: Got 116 measurments
754: Got 115 measurments
755: Got 116 measurments
756: Got 117 measurments
757: Got 116 measurments
758: Got 115 measurments
759: Got 115 measurments
760: Got 114 measurments
761: Got 118 measurments
762: Got 112 measurments
763: Got 114 measurments
764: Got 112 measurments
765: Got 114 measurments
766: Got 113 measurments
767: Got 115 measurments
768: Got 110 measurments
769: Got 114 measurments
X: 220, Y: 767
[[244.8688079  767.1122612 ]
 [175.84067085 779.63506242]
 [192.75423466 725.14236112]
 [229.99083468 815.40255924]
 [258.31115518 748.49618358]]
770: Got 117 measurments
771: Got 114 measurments
772: Got 111 measurments
773: Got 112 measurments
774: Got 113 measurments
X: 199, Y: 736
[[185.71807495 743.78279231]
 [211.61387264 733.2

Too many measurments in the input buffer: 502/500. Clearing buffer...


775: Got 114 measurments
776: Got 116 measurments
777: Got 114 measurments
778: Got 116 measurments
779: Got 116 measurments
780: Got 115 measurments
781: Got 115 measurments
782: Got 115 measurments
783: Got 117 measurments
784: Got 114 measurments
785: Got 114 measurments
786: Got 112 measurments
787: Got 115 measurments
788: Got 112 measurments
789: Got 111 measurments
790: Got 116 measurments
791: Got 115 measurments
792: Got 111 measurments
793: Got 114 measurments
794: Got 116 measurments
X: -378, Y: 332
[[-329.53362609  373.95467104]
 [-406.92631899  370.56392752]
 [-398.84157097  279.90785145]
 [-342.5902802   317.61114744]
 [-412.5813574   318.42770852]]
795: Got 114 measurments
796: Got 116 measurments
797: Got 116 measurments
798: Got 114 measurments
799: Got 115 measurments
800: Got 115 measurments
801: Got 115 measurments
802: Got 115 measurments
803: Got 118 measurments
804: Got 115 measurments
805: Got 115 measurments
806: Got 114 measurments
807: Got 113 measurments
808